In [10]:
import os
import sys
import json

with open(".\settings.json", "r") as jsonfile:
    settings = json.load(jsonfile)
    print(settings)

{'ACCOUNT_REGION': 'eastus', 'ACCOUNT_KEY': '4f47c6853d074715925ff2c1c957fe52'}


In [3]:
pip install azure


  Using cached azure-5.0.0.zip (4.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [25 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\ACER\AppData\Local\Temp\pip-install-jto7k62e\azure_14ac187bfc614810b77360a67766ac7c\setup.py", line 60, in <module>
          raise RuntimeError(message)
      RuntimeError:
      
      Starting with v5.0.0, the 'azure' meta-package is deprecated and cannot be installed anymore.
      Please install the service specific packages prefixed by `azure` needed for your application.
      
      The complete list of available packages can be found at:
      https://aka.ms/azsdk/python/all
      
      Here's a non-exhaustive list of common packages:
      
      -  azure-mgmt-compute (https://pypi.python.org/pypi/azure-mgmt-compute) : Management of Virtual Machines, etc.
      -  azure-

In [6]:
pip install azure-cognitiveservices-vision-computervision

   ---------------------------------------- 0.0/85.4 kB ? eta -:--:--
   -------------------------------------- - 81.9/85.4 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 85.4/85.4 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/193.4 kB ? eta -:--:--
   ------------------------- -------------- 122.9/193.4 kB 3.6 MB/s eta 0:00:01
   -------------------------------------- - 184.3/193.4 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 193.4/193.4 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/41.7 kB ? eta -:--:--
   ---------------------------------------- 41.7/41.7 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/151.7 kB ? eta -:--:--
   ------------------------ --------------- 92.2/151.7 kB ? eta -:--:--
   ---------------------------------------- 151.7/151.7 kB 4.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [11]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

region = settings["ACCOUNT_REGION"]
key = settings["ACCOUNT_KEY"]

credentials = CognitiveServicesCredentials(key)
client = ComputerVisionClient(
    endpoint="https://ocr-track-wallet.cognitiveservices.azure.com/",
    credentials=credentials
)

In [22]:
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
import time

def extract_text_from_image(image_path):
    with open(image_path, "rb") as image:
        rawHttpResponse = client.read_in_stream(image, language="id", raw=True)
        operation_location = rawHttpResponse.headers["Operation-Location"]
        id_location = len(operation_location) - 36
        operation_id = operation_location[id_location:]

        while True:
            result = client.get_read_result(operation_id)
            if result.status not in ['notStarted', 'running']:
                break
            time.sleep(1)

    return result
    

In [23]:
result = extract_text_from_image("./image/tes.jpeg")

In [24]:
result.status

<OperationStatusCodes.succeeded: 'succeeded'>

In [95]:
receipt1 = ''
if result.status == OperationStatusCodes.succeeded:
    for line in result.analyze_result.read_results[0].lines:
        print(line.text)
        receipt1 += line.text + '\n'

MIXUE
SINCE 1997-ICE CREAM & TEA
DINE IN#138
Kaliurang
Nomor nota: 240518135710038
Waktu pemesanan: 2024-05-18 13:57:56
Nama
harga satuan Kuantitas
Subtotal
.Brown Sugar Pearl Milk Tea(Tall)
21,000
1
21,000
Atribut: normal ice,normal sugar
Pesch Tea
18,000
1
18,000
Atribut: normal ice, normal sugar
.Lemon Jasmine Tea
14,000
1
14,000
Atribut: less ice, less sugar
Coconut Jelly Milk Tea(Grande)
24,000
1
24,000
Atribut: normal ice,less sugar
.Oreo Sundae
18,000
1
18,000
Atribut: normal ice,normal sugar
Total
5
95,000
1.GoFood
95,000
Kasir: 80220145903
Alamat: Ji. Kaliurang No.81 Km 5 Kocoran, Catur
tunggal, Kec. Depok, Kabupaten Sleman, Daerah
Istimewa Yogyakarta
Waktu: 2024-05-18 13:57:59


In [ ]:
# result2 = extract_text_from_image("./image/tes2.jpg")
result2.status
receipt2 = ''
if result2.status == OperationStatusCodes.succeeded:
    for line in result2.analyze_result.read_results[0].lines:
        print(line.text)
        receipt2 += line.text + '\n'


In [105]:
result3 = extract_text_from_image("./image/tes3.jpeg")
result3.status
receipt3 = ''
if result3.status == OperationStatusCodes.succeeded:
    for line in result3.analyze_result.read_results[0].lines:
        print(line.text)
        receipt3 += line.text + '\n'


COTO DG. STRUA
ASUHAN HJ. SINGARA
JL. ABD. DG. SIRUA NO. 10
#000007 24/07/2015
11:29
01 CLERRO1
000000
1× 15000
RR15000
COTO
1x 5000
TEH MANIS
Rp5000
1× 1000
Rp1000
KETUPAT
###TOTAL
RP21000
BAYAR
Rp21000
KEMBALI
Re
TERIMA KASIH
ATAS KUNJUNGAN ANDA


In [103]:
import re

def extract_items_and_prices(receipt_text):
    items = []
    
    # Define regex patterns for matching item names and prices
    item_pattern = re.compile(r'\.?(.*?)\n')
    price_pattern = re.compile(r'Rp?*(\d{1,3}(?:,\d{3})*)')

    # Find all item names and prices in the receipt text
    item_matches = item_pattern.findall(receipt_text)
    price_matches = price_pattern.findall(receipt_text)

    # Remove non-item lines from item_matches
    item_matches = [item for item in item_matches if any(char.isalpha() for char in item)]

    # Convert price matches to integers
    price_matches = [int(price.replace(',', '')) for price in price_matches]

    # Create the items array
    for item, price in zip(item_matches, price_matches):
        # items.append({'item': item.strip(), 'price': price})
        items = ({'item': item.strip(), 'price': price })
    
    return items

In [85]:
receipt2

'F10F1NurPERTAMINA8%34.40111J1.CIPAGANTI, PASTEUR59SUKAJADI, BANDUNGShift : 2No Trans: 273512Waktu : 27/10/2021 16:03:415Pulau/Pompa :11Nama Produk :PERTAMAXHarga/Liter :Rp. 9,000Volume(L) 27.77K..Total Harga :Rp. 250,000Operator:TITINCASH250,0000AYO PATUHI 5M : MEMAKAI MASKER,MENCUCI TANGAN, MENJAGA JARAK,MEMBATASI KERUMUNAN & MOBILITASTERIMA KASIH'

In [106]:
items1 = extract_items_and_prices(receipt1)
items2 = extract_items_and_prices(receipt2)
items3 = extract_items_and_prices(receipt3)

print(items1)
print(items2)
print(items3)

[]
{'item': 'F1', 'price': 250000}
[]
